In [1]:
import requests
from psycopg2.extras import execute_values
from datetime import datetime, timezone
from ipyleaflet import Map, LayerGroup, Marker, DivIcon, FullScreenControl, basemaps, TileLayer
from ipywidgets import Layout
import psycopg2
import time
from itertools import cycle

DB_CONFIG = {
    "host": "localhost", "port": "5432",
    "database": "flight_data", "user": "admin", "password": "admin"
}

plane_svg = """
<svg width="30px" height="30px" viewBox="0 0 512 512" version="1.1" xmlns="http://www.w3.org/2000/svg" style="transform: rotate({heading}deg);">
    <path fill="{color}" d="M448 336v-40L288 192V79.2c0-22.1-17.9-40-40-40s-40 17.9-40 40V192L48 296v40l160-48v113.6l-48 31.2V472l88-24 88 24v-39.2l-48-31.2V288l160 48z" stroke="white" stroke-width="2"/>
</svg>
"""

In [2]:
def initialize_database():
    conn = psycopg2.connect(
        host="localhost",
        port="10000",
        database="flight_data",
        user="admin",
        password="admin"
    )
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    print("Initializing Database...")
    cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
    cur.execute("CREATE EXTENSION IF NOT EXISTS timescaledb;")
    cur.execute("""
    CREATE TABLE IF NOT EXISTS observations (
        time        TIMESTAMPTZ NOT NULL,
        icao24      VARCHAR(10) NOT NULL,
        callsign    VARCHAR(20),
        location    GEOMETRY(POINT, 4326),
        altitude    DOUBLE PRECISION,
        velocity    DOUBLE PRECISION,
        heading     DOUBLE PRECISION,
        on_ground   BOOLEAN
    );
    """)
    try:
        cur.execute("SELECT create_hypertable('observations', 'time');")
        print("Hypertable created successfully.")
    except psycopg2.errors.DuplicateObject:
        print("Hypertable already exists, skipping.")
    cur.execute("CREATE INDEX IF NOT EXISTS idx_icao_time ON observations (icao24, time DESC);")

    cur.close()
    conn.close()
    print("Database is ready!")

In [3]:
def check_row_count():
    conn = psycopg2.connect(host="localhost", database="flight_data", user="admin", password="admin", port=10000)
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM observations;")
    count = cur.fetchone()[0]
    print(f"Current rows in DB: {count}")
    cur.close()
    conn.close()

In [4]:
def save_to_db(planes):
    # Connection details from your docker-compose
    conn = psycopg2.connect(
        host="localhost",
        port="10000",
        database="flight_data",
        user="admin",
        password="admin"
    )
    cur = conn.cursor()
    execute_values(cur, """
        INSERT INTO observations (time, icao24, callsign, location, altitude, velocity, heading, on_ground)
        VALUES %s""",
        [(datetime.fromtimestamp(p['time'], tz=timezone.utc), p['icao24'], p['callsign'], f"SRID=4326;POINT({p['lon']} {p['lat']})",
          p['altitude'], p['velocity'], p['heading'], p['on_ground']) for p in planes])

    conn.commit()
    cur.close()
    conn.close()
    print(f"Successfully saved {len(planes)} rows to TimescaleDB.")

In [5]:
def get_live_planes(username=None, password=None):
    # API Endpoint for all states
    url = "https://opensky-network.org/api/states/all"

    # Optional: Use auth to get higher rate limits (4000/day vs 100/day)
    auth = (username, password) if username and password else None

    try:
        response = requests.get(url, auth=auth, timeout=10)
        response.raise_for_status()
        data = response.json()

        states = data.get('states', [])
        timestamp = data.get('time')

        processed_planes = []

        for s in states:
            # We filter for planes that have valid lat/lon
            if s[5] is not None and s[6] is not None:
                processed_planes.append({
                    "time": timestamp,
                    "icao24": s[0],
                    "callsign": s[1].strip() if s[1] else "NONE",
                    "lon": s[5],
                    "lat": s[6],
                    "altitude": s[7], # Barometric altitude
                    "velocity": s[9],
                    "heading": s[10] or 0, # Default to 0 if null
                    "on_ground": s[8]
                })

        return processed_planes

    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

In [ ]:
# planes = get_live_planes()
# save_to_db(planes)
# print(f"Found {len(planes)} planes with GPS data.")
# check_row_count()

In [ ]:
# clean_old_data()

In [6]:
def clean_old_data():
    """Optional: Curata tabelul pentru a avea un demo clar."""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("TRUNCATE observations;") # Atentie! Sterge tot.
    conn.commit()
    conn.close()
    print("Baza de date a fost golită pentru test.")

def save_batch(planes):
    if not planes: return
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    query = """
        INSERT INTO observations (time, icao24, callsign, location, altitude, velocity, heading, on_ground)
        VALUES %s ON CONFLICT DO NOTHING
    """
    values = [
        (datetime.fromtimestamp(p['time'], tz=timezone.utc), p['icao24'], p['callsign'],
         p['lon'], p['lat'], p['altitude'], p['velocity'], p['heading'], p['on_ground'])
        for p in planes
    ]
    # Folosim template pentru PostGIS geometry
    execute_values(cur, query, values, template="""
    (%s, %s, %s, ST_SetSRID(ST_MakePoint(%s, %s), 4326), %s, %s, %s, %s)
    """)
    conn.commit()
    conn.close()

# --- Executia ---
# clean_old_data() # Decomenteaza daca vrei sa stergi datele anterioare
#
# ITERATIONS = 5
# FETCH_INTERVAL = 10 # Interval de colectare (secunde)
#
# print(f"Începem înregistrarea a {ITERATIONS} cadre...")
#
# for i in range(1, ITERATIONS + 1):
#     loop_start = time.time()
#
#     # 1. Fetch (Foloseste functia ta existenta get_live_planes)
#     planes = get_live_planes()
#     print(f"[{i}/{ITERATIONS}] Capturat {len(planes)} avioane.")
#
#     # 2. Save
#     save_batch(planes)
#
#     # 3. Wait (pentru a respecta intervalul de colectare)
#     elapsed = time.time() - loop_start
#     time.sleep(max(0, FETCH_INTERVAL - elapsed))
#
# print("✅ Înregistrare finalizată!")

In [ ]:
m = Map(
    center=(51.5, -0.1),
    zoom=7,
    scroll_wheel_zoom=True,
    world_copy_jump=False,
    basemap=basemaps.OpenStreetMap.Mapnik,
    layout=Layout(width='100%', height='600px')
)

plane_layer = LayerGroup()
m.add_layer(plane_layer)
m.add_control(FullScreenControl())
display(m)

def get_all_timestamps():
    """Luam toate momentele de timp disponibile."""
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("SELECT DISTINCT time FROM observations ORDER BY time ASC")
    rows = cur.fetchall()
    conn.close()
    return [row[0] for row in rows]

def get_planes_sync(timestamp, bounds):
    """Query simplu si direct."""
    # Extragem coordonatele. Daca bounds e gol (la start), punem Europa default
    if not bounds:
        south, west, north, east = 35.0, -10.0, 60.0, 40.0
    else:
        ((south, west), (north, east)) = bounds

    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    query = """
    SELECT icao24, callsign, ST_Y(location::geometry) as lat, ST_X(location::geometry) as lon, heading, on_ground
    FROM observations
    WHERE time = %s
    AND location && ST_MakeEnvelope(%s, %s, %s, %s, 4326)
    LIMIT 100; -- Limita ceruta de tine
    """

    cur.execute(query, (timestamp, west, south, east, north))
    data = cur.fetchall()
    conn.close()
    return data

def run_sync_animation():
    nr_iter = 0
    print("Se încarcă timestamp-urile...")
    timestamps = get_all_timestamps()

    if not timestamps:
        print("Nu există date în DB.")
        return

    print(f"Start animație sincronă ({len(timestamps)} cadre).")
    print("Pentru a opri, apasă butonul 'Interrupt Kernel' (pătratul) din meniul de sus.")

    # Cache simplu pentru markeri ca sa nu flickereasca rau
    active_markers = {}

    pool = cycle(timestamps)

    for ts in pool:
        try:
            if nr_iter == 20:
                break

            # 1. Luam bounds CURENT (Direct din harta)
            current_bounds = m.bounds
            with open("logger", "a") as f:
                f.write(f"{nr_iter}: bounds: {current_bounds}\n")

            # 2. Facem request la DB
            planes = get_planes_sync(ts, current_bounds)
            print(current_bounds)
            print(f"planes = {len(planes)}")

            current_frame_icaos = set()

            # 3. Construim markerii
            for p in planes:
                icao, callsign, lat, lon, heading, on_ground = p
                current_frame_icaos.add(icao)

                color = "#28a745" if on_ground else "#007bff"
                icon_html = plane_svg.format(heading=heading or 0, color=color)
                new_icon = DivIcon(html=icon_html, bg_pos=[0, 0], icon_size=[30, 30])

                if icao in active_markers:
                    # Update
                    marker = active_markers[icao]
                    marker.location = (lat, lon)
                    marker.icon = new_icon
                else:
                    # Create
                    marker = Marker(location=(lat, lon), icon=new_icon, title=callsign or icao, draggable=False)
                    active_markers[icao] = marker

            # 4. Stergem ce nu mai e in lista de 100
            keys_to_remove = [k for k in active_markers if k not in current_frame_icaos]
            for k in keys_to_remove:
                del active_markers[k]

            # 5. Afisam
            plane_layer.layers = tuple(active_markers.values())

            # 6. Pauza (Blocheaza executia)
            nr_iter += 1
            time.sleep(2)

        except KeyboardInterrupt:
            print("Oprit de utilizator.")
            return
        except Exception as e:
            print(f"Eroare: {e}")
            time.sleep(2)

# Pornim direct
run_sync_animation()

Map(center=[51.5, -0.1], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

Se încarcă timestamp-urile...
Start animație sincronă (5 cadre).
Pentru a opri, apasă butonul 'Interrupt Kernel' (pătratul) din meniul de sus.
()
planes = 100
()
planes = 100
()
planes = 100
()
planes = 100
()
planes = 100
()
planes = 100
()
planes = 100
()
planes = 100
()
planes = 100
()
planes = 100
()
planes = 100
Oprit de utilizator.
